<a href="https://colab.research.google.com/github/dsantasantos/techchallengeFase3/blob/main/FineTuning_TC3_V001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-52m5bhun/unsloth_011757c22be14bbba5e2ac3d1e908903
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-52m5bhun/unsloth_011757c22be14bbba5e2ac3d1e908903
  Resolved https://github.com/unslothai/unsloth.git to commit 9645beb7e1c57cecb06ff0a1e85008a673ad1829
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.6/180.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 38.9 MB/s eta 0:00:00


In [2]:
import os
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
from datasets import Dataset
import xml.etree.ElementTree as ET
import pandas as pd

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.12.5: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Antes do treino
def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda:0")
    outputs = model.generate(**inputs, max_new_tokens=20)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

print("Antes do treino\n")
generate_text("<human>: What is (are) Rectal Cancer ?\n<bot>: ")

Antes do treino

<human>: What is (are) Rectal Cancer?
<bot>: 1. A malignant tumor of the rectum.
2. A malignant tumor of the rectum.



In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = 16,
    lora_dropout = 0,  # Supports any, but = 0 is optimized
    bias = "none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
    use_rslora = False,   # Rank stabilized LoRA
    loftq_config = None,  # LoftQ
)

Unsloth 2025.12.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [10]:
tree = ET.parse("/content/drive/MyDrive/06-ESTUDOS/FIAP/IA PARA DEVS/Fase 3 - Aulas/0000037_2.xml")
root = tree.getroot()

data = []

focus = root.find("Focus")
focusText = focus.text

for qa in root.find("QAPairs").findall("QAPair"):
    pid = qa.get("pid")

    question_tag = qa.find("Question")
    question = question_tag.text.strip()
    qid = question_tag.get("qid")
    qtype = question_tag.get("qtype")

    answer = qa.find("Answer").text.strip()

    data.append({
        "pid": pid,
        "focus": focusText,
        "qid": qid,
        "qtype": qtype,
        "question": question,
        "answer": answer.replace("Key Points\n", "")
    })

df = pd.DataFrame(data)

from datasets import Dataset
dataset = Dataset.from_pandas(df)

def formatting_func(examples):
    texts = []
    for i in range(len(examples["question"])):
        question = examples["question"][i].strip()
        answer   = examples["answer"][i].strip()
        texts.append(f"<human>: {question}\n<bot>: {answer}")
    return texts

In [11]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    formatting_func = formatting_func,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Disable wandb
    ),
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/8 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8 | Num Epochs = 60 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.812100
2,0.812100
3,0.810800
4,0.803900
5,0.785500
6,0.754200
7,0.714700
8,0.672800
9,0.632200
10,0.593900


TrainOutput(global_step=60, training_loss=0.26946258284151553, metrics={'train_runtime': 899.3078, 'train_samples_per_second': 0.534, 'train_steps_per_second': 0.067, 'total_flos': 1.4508007927136256e+16, 'train_loss': 0.26946258284151553, 'epoch': 60.0})

In [12]:
print("Depois do treino\n")
generate_text("<human>: What is (are) Rectal Cancer ?\n<bot>: ")

Depois do treino

<human>: What is (are) Rectal Cancer?
<bot>:  Rectal cancer is a disease in which malignant (cancer) cells form in the tissues of the
